# PyGnocchi

Statistical associations using the ADAM genomics analysis platform. The currently supported operations are Genome Wide Association using Linear and Logistic models with either Dominant or Additive assumptions.

## Installation and Usage

(TODO) Make this a pip package

Download the Gnocchi [source code](https://github.com/nathanielparke/gnocchi) and package the project
``` mvn package ```

Start a virtual environment and build the Python files
``` 
virtualenv gnocchi
. gnocchi/bin/activate
mvn -Ppython package
```

Start a Jupyter notebook by running the Pygnocchi script
```
. bin/pygnocchi-notebook
```

## Using GnocchiSession

In [1]:
from bdgenomics.gnocchi.gnocchiSession import GnocchiSession
from bdgenomics.gnocchi.linearGnocchiModel import LinearGnocchiModel
from bdgenomics.gnocchi.logisticGnocchiModel import LogisticGnocchiModel
from bdgenomics.gnocchi.regressPhenotypes import RegressPhenotypes

In [2]:
genotypesPath = "../examples/testData/1snp10samples.vcf"
phenotypesPath = "../examples/testData/10samples1Phenotype.txt"

## Create GnocchiSession in Python

`GnocchiSession` handles a lot of the pipelining functionality with regards to loading and preparing raw genotype and phenotype data

In [3]:
gs = GnocchiSession(spark)

In [4]:
# Returns CalledVariantDataset which is a Python wrapper
# around a Scala Dataset[CalledVariant]
genos = gs.loadGenotypes(genotypesPath)
phenos = gs.loadPhenotypes(phenotypesPath, "SampleID", "pheno1", "\t")

## Build a LinearGnocchiModel

We can use the loaded genotypes and phenotypes to build a GnocchiModel which packages all the GWAS outputs and be merged with other models.

In [5]:
lgm = LinearGnocchiModel.New(spark, genos, phenos, ["AD"], ["GI"])

## Regress Phenotypes on full data

While GnocchiModels do provide functionality for packaging the operations in a portable fashion, in order to directly see GWAS outputs use `RegressPhenotypes`. This essentially takes a string of arguments canonical with regular Gnocchi command line flags and runs the specified regression.

In [6]:
rp = RegressPhenotypes(spark)
rp.apply("../examples/testData/1snp10samples.vcf ../examples/testData/10samples1Phenotype.txt ADDITIVE_LINEAR ../examples/testData/DELETEME -saveAsText -sampleIDName SampleID -phenoName pheno1 -overwriteParquet")

In [1]:
# Verify that the Python run regression is concordant 
# with the results of the gnocchi CLI 
! bash ../bin/gnocchi-submit regressPhenotypes ../examples/testData/1snp10samples.vcf ../examples/testData/10samples1Phenotype.txt ADDITIVE_LINEAR ../examples/testData/DELETEME2 -saveAsText -sampleIDName SampleID -phenoName pheno1 -overwriteParquet

In [2]:
# Verify the files are identifical
! diff ../examples/testData/DELETEME/part-00000-fab16dec-e163-448c-a5f5-be921bf52584-c000.csv ../examples/testData/DELETEME2/part-00000-8cf524fc-b4a4-43a2-85b7-26822a089110-c000.csv

## Filter out variants

In addition to just loading genotypes and phenotypes, GnocchiSession can also filter variants and samples. The API is the same for the Scala shell and we verify that the Datasets output have reasonable properties.

In [9]:
filteredGenos = gs.filterVariants(genos, 0.0, 0.5)
unfilteredGenos = gs.filterVariants(genos, 0.0, 0.0)

In [10]:
assert genos.get().count() != filteredGenos.get().count(), "Counts are same"
assert filteredGenos.get().count() == 0, "All items not filtered out"

In [11]:
assert genos.get().count() == unfilteredGenos.get().count(), "Counts are same"

## Another example with filter samples

Here we replicate the Gnocchi Scala example involving the time dataset (`time_genos_1.vcf` and `tab_time_phenos.txt`) as it demonstrates the full suite of GnocchiSession functionality. We load phenotypes and genotypes, we then filter the samples and pass those into a filter by variant. We can access the underlying Dataset and verify the output.

In [12]:
genotypesPath = "../examples/testData/time_genos_1.vcf"
phenotypesPath = "../examples/testData/tab_time_phenos_1.txt"

In [13]:
geno = gs.loadGenotypes(genotypesPath)
pheno = gs.loadPhenotypes(phenotypesPath, "IID", "pheno_1", "\t", phenotypesPath, ["pheno_4", "pheno_5"])

In [14]:
filteredGenos = gs.filterSamples(geno, 0.1, 2)
filteredGenosVariants = gs.filterVariants(filteredGenos, 0.1, 0.1)

In [15]:
assert filteredGenosVariants.get().head().position() == 75094266

## Accessing Dataset operations

Of particular convenience is the fact that we can run Scala dataset operations on the Python-wrapped CalledVariantDataset objects, because the underlying Datasets have been exposed from the JVM to Python

In [24]:
calledVariant = filteredGenosVariants.get().head()

In [3]:
print("calledVariant.chromosome =", calledVariant.chromosome())
print("calledVariant.position =", calledVariant.position())
print("calledVariant.uniqueID =", calledVariant.uniqueID())
print("calledVariant.referenceAllele =", calledVariant.referenceAllele())
print("calledVariant.alternateAllele =", calledVariant.alternateAllele())